In [ ]:
# run IPYNB that imports all the relevant packages and functions
%run ../ms_packages_and_functions.ipynb

In [ ]:
output_dir = pjoin(os.getcwd(), 'processed_data')
data_dir   = pjoin(os.getcwd(), 'raw_data')
fig_dir    = pjoin(os.getcwd(), 'figures')


In [ ]:
data = loadPickle('raw_data/rockstar_CenterOut_data_10msBin_with500buffer.pkl')

In [ ]:
rand_state = '000' #np.random.randint(1000000000)

In [ ]:
folder_path = pjoin(output_dir,'RS_nulls-poisson-200ms-{}'.format(rand_state))

if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    print("created directory: {}".format(folder_path))
else:
    print("All files will be saved in: {}".format(folder_path))


# 1. Example of Inhomogenous Poisson Model

In [ ]:
# run = True

In [ ]:
buffer = 0.5
plt.figure()
num_sim = 50
num_units = 10
kernel_sigma = 20*pq.ms
sampling_period = 10*pq.ms
direction = 0
trial = 10
kernel = el.kernels.GaussianKernel(sigma = kernel_sigma)

# if run:
#     for unit in range(0,num_units):
#         eventplot_data = []
#         eventplot_c = []
#         t_start = data[direction]['trialData'][trial]['trialStart']-buffer
#         t_stop  =  data[direction]['trialData'][trial]['trialEnd']+buffer - t_start

#         spiketimes = np.array(data[direction]['trialData'][trial]['trialSpiketimes']['spiketimes'][unit])-t_start
#         spiketrain = neo.core.SpikeTrain(spiketimes*pq.s,\
#                         t_start = 0,\
#                         t_stop = t_stop)

#         kernel = el.kernels.GaussianKernel(sigma = kernel_sigma)

#         fr = el.statistics.instantaneous_rate(spiketrain,sampling_period=sampling_period,kernel=kernel)
#         eventplot_data.append(spiketrain.simplified.magnitude)
#         eventplot_c.append('black')

#         plt.figure(figsize=(7,5))
#         plt.plot(fr.times.simplified.magnitude, fr.magnitude, color='b',linewidth=2,label='data')

#         poiss_fr = []
#         for x in range(0,num_sim):
#             poiss = el.spike_train_generation.inhomogeneous_poisson_process(fr)
#             poiss_fr_temp = el.statistics.instantaneous_rate(poiss,sampling_period=sampling_period,kernel=kernel)
#             poiss_fr.append(poiss_fr_temp)
#             eventplot_data.append(poiss.simplified.magnitude)
#             eventplot_c.append('green')     
#         poiss_fr = np.mean(np.array(poiss_fr),axis=0)
#         max_fr = np.max([np.max(poiss_fr),np.max(fr.simplified.magnitude)])
#         plt.plot(poiss_fr_temp.times.simplified.magnitude, poiss_fr,'r',linewidth=2,label='poisson')

#         plt.eventplot(eventplot_data,color=eventplot_c,lineoffsets=(max_fr/num_sim)*1.001, linelengths=(max_fr/num_sim),alpha=0.5)
#         plt.legend(frameon=False)
#         plt.title("UNIT {}: Firing Rate and Raster for Data and Poisson Model, n={}".format(unit,num_sim))
#         plt.xlabel('time (s)')
#         plt.ylabel('FR ($s^{-1}$)')
#         ax = plt.gca()
#         ax.spines['right'].set_visible(False)
#         ax.spines['top'].set_visible(False)
#         ax.spines['bottom'].set_visible(False)
#         ax.spines['left'].set_visible(False)
        
#         save_dir = pjoin(folder_path,'poisson-vs-data_unit{}_direction{}_trial{}.png'.format(str(unit).zfill(3),direction,trial))
#         plt.savefig(save_dir)

# 2. Generate FNs: 200 ms moving window

In [ ]:
run = True

In [ ]:
if run:
    mov_window = 0.2 # 200 ms
    binwin = data[0]['binwin']
    mov_10ms = int(0.01/binwin)
    buffer = 0.5

    num_units = data[0]['trialData'][0]['trialNumUnits'] + 1

    num_sim = 10


    for sim in range(0,num_sim):
        poisson_rasters = []
        poisson_FNs = []

        direction_rasters = []
        direction_FNs = []
        for direction in range(0,len(data)):

            windowBin = int(mov_window/binwin)
            direction_rasters.append([])
            direction_FNs.append([])

            trial_rasters  = []
            trial_FNs = []
            
            for trial in range(0,len(data[direction]['trialData'])):


                trial_rasters.append([])
                trial_FNs.append([])

                poiss = {}
                poiss['spiketimes'] = []

                # start time at 0
                t_start = data[direction]['trialData'][trial]['trialStart']-buffer
                t_stop  =  data[direction]['trialData'][trial]['trialEnd']+buffer - t_start
                

                for unit in range(0,num_units):
                    status("> Generating Poisson Spike Trains for direction: {}/{},trial: {}/{}, unit: {}/{}".format(direction, len(data), \
                            trial,len(data[direction]['trialData']), unit,num_units))
                    spiketimes = np.array(data[direction]['trialData'][trial]['trialSpiketimes']['spiketimes'][unit])-t_start
                    spiketrain = neo.core.SpikeTrain(spiketimes*pq.s,\
                                    t_start = 0,\
                                    t_stop = t_stop)
                    fr = el.statistics.instantaneous_rate(spiketrain,sampling_period=sampling_period,kernel=kernel)
                    
                    poiss_spiketrain = el.spike_train_generation.inhomogeneous_poisson_process(fr)

                    poiss['spiketimes'].append(poiss_spiketrain.times.simplified.magnitude)
                    
                trialData,_ = bin_spikes(poiss,0,t_stop,binwin,plot=False)

                time_FNs = []
                for j in range(0,trialData[0].shape[0]-windowBin,mov_10ms):
                    status("> Computing FN for direction: {},trial: {}, window: {}/{}".format(direction, trial,j/mov_10ms,(trialData[0].shape[0]-windowBin-1)/mov_10ms))

                    temp = [x[j:j+windowBin] for x in trialData]
                    tempFN = make_FN(temp,metric='fMI',plot=False)
                    time_FNs.append(tempFN)
                trial_FNs[trial].append(time_FNs)
                trial_rasters[trial].append(trialData)

            direction_rasters[direction].append(trialData)
            direction_FNs[direction].append(trial_FNs)

        poisson_rasters.append(direction_rasters)
        poisson_FNs.append(direction_FNs)

        print('> SAVING...')
        save_dir = pjoin(folder_path,'poisson_rasters_{}.pkl'.format(str(sim).zfill(3)))
        savePickle(save_dir,poisson_rasters)
        print('>> saved poisson_rasters in folder: {}'.format(save_dir))
        

        save_dir = pjoin(folder_path,'poisson_FNs_{}.pkl'.format(str(sim).zfill(3)))
        savePickle(save_dir,poisson_FNs)
        print('>> saved poisson_FNs in folder: {}'.format(save_dir))

        del poisson_rasters,poisson_FNs, direction_rasters, direction_FNs, trial_rasters, trial_FNs
